**Neural Network - Machine 99**

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
import numpy as np

import tensorflow as tf
import datetime as dt

In [19]:
# Import our input dataset
all_tele_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Resources/all_tele_main.csv')
all_tele_df = all_tele_df.sort_values(by=['datetime']).drop_duplicates(subset=['datetime', 'machineid'], keep='first').reset_index(drop=True)
all_tele_df.head()

<ipython-input-19-51b3cfc196db>:2: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  all_tele_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Resources/all_tele_main.csv')


,datetime,machineid,machinemodel,machineage,volt,rotate,pressure,vibration,errorid,compmaint,compfail
0,2015-01-01 06:00:00,34,model4,10,181.859394,483.215401,82.463735,37.849856,0,comp2,0
1,2015-01-01 06:00:00,41,model4,9,162.314459,424.540949,129.113722,52.591978,0,0,0
2,2015-01-01 06:00:00,39,model4,0,150.138449,421.747438,100.174935,40.367037,0,0,0
3,2015-01-01 06:00:00,10,model3,10,158.421261,500.830885,119.750673,45.571344,0,0,0
4,2015-01-01 06:00:00,45,model3,14,191.649670,383.200891,85.630577,52.169106,0,0,0


In [20]:
#check data types
all_tele_df.dtypes

datetime         object
machineid         int64
machinemodel     object
machineage        int64
volt            float64
rotate          float64
pressure        float64
vibration       float64
errorid          object
compmaint        object
compfail          int64
dtype: object

In [21]:
# update date to integer ready for testing
all_tele_df['datetime'] = pd.to_datetime(all_tele_df['datetime'])
all_tele_df['datetime'] = all_tele_df['datetime'].astype(np.int64)
all_tele_df.dtypes

datetime          int64
machineid         int64
machinemodel     object
machineage        int64
volt            float64
rotate          float64
pressure        float64
vibration       float64
errorid          object
compmaint        object
compfail          int64
dtype: object

In [22]:
# filter for Machine 99 only
machine_99 = all_tele_df.loc[(all_tele_df['machineid'] == 99)].drop(columns=['machinemodel'])
machine_99.head()

,datetime,machineid,machineage,volt,rotate,pressure,vibration,errorid,compmaint,compfail
34,1420092000000000000,99,14,168.596133,384.747105,110.921131,41.944692,0,0,0
192,1420095600000000000,99,14,153.667693,441.288719,128.011446,39.271527,0,0,0
261,1420099200000000000,99,14,178.319255,512.612661,128.526147,42.975412,0,0,0
363,1420102800000000000,99,14,162.163821,493.356626,114.435754,40.044016,0,0,0
418,1420106400000000000,99,14,180.201336,550.512296,110.771891,43.054530,0,0,0


In [23]:
# use get_dummies to convert categorical variables
machine_99 = pd.get_dummies(data=machine_99, columns=['errorid', 'compmaint'])
machine_99.head()

,datetime,machineid,machineage,volt,rotate,pressure,vibration,compfail,errorid_0,errorid_0,...,errorid_error2,errorid_error3,errorid_error4,errorid_error5,compmaint_0,compmaint_0,compmaint_comp1,compmaint_comp2,compmaint_comp3,compmaint_comp4
34,1420092000000000000,99,14,168.596133,384.747105,110.921131,41.944692,0,0,1,...,0,0,0,0,1,0,0,0,0,0
192,1420095600000000000,99,14,153.667693,441.288719,128.011446,39.271527,0,0,1,...,0,0,0,0,0,1,0,0,0,0
261,1420099200000000000,99,14,178.319255,512.612661,128.526147,42.975412,0,0,1,...,0,0,0,0,0,1,0,0,0,0
363,1420102800000000000,99,14,162.163821,493.356626,114.435754,40.044016,0,0,1,...,0,0,0,0,1,0,0,0,0,0
418,1420106400000000000,99,14,180.201336,550.512296,110.771891,43.054530,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [24]:
# split preprocessed data into feature and target arrays
y = machine_99.compfail.values
X = machine_99.drop(columns='compfail').values

# split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [25]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 50 ## updated but not run yet
hidden_nodes_layer2 = 25 #3 updated but not run yet

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                1050      
                                                                 
 dense_1 (Dense)             (None, 25)                1275      
                                                                 
 dense_2 (Dense)             (None, 1)                 26        
                                                                 
Total params: 2351 (9.18 KB)
Trainable params: 2351 (9.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
206/206 [==============================] - 2s 3ms/step - loss: 0.1349 - accuracy: 0.9960
Epoch 2/100
206/206 [==============================] - 1s 3ms/step - loss: 0.0305 - accuracy: 0.9985
Epoch 3/100
206/206 [==============================] - 1s 3ms/step - loss: 0.0078 - accuracy: 0.9986
Epoch 4/100
206/206 [==============================] - 1s 3ms/step - loss: 0.0050 - accuracy: 0.9986
Epoch 5/100
206/206 [==============================] - 0s 2ms/step - loss: 0.0035 - accuracy: 0.9991
Epoch 6/100
206/206 [==============================] - 0s 2ms/step - loss: 0.0044 - accuracy: 0.9989
Epoch 7/100
206/206 [==============================] - 0s 2ms/step - loss: 0.0041 - accuracy: 0.9986
Epoch 8/100
206/206 [==============================] - 0s 2ms/step - loss: 0.0027 - accuracy: 0.9991
Epoch 9/100
206/206 [==============================] - 0s 2ms/step - loss: 0.0037 - accuracy: 0.9988
Epoch 10/100
206/206 [==============================] - 0s 2ms/step - loss: 0.0025 - accura

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

69/69 - 1s - loss: 6.7433e-06 - accuracy: 1.0000 - 698ms/epoch - 10ms/step
Loss: 6.743297490174882e-06, Accuracy: 1.0


In [30]:
# Making predictions using the testing data
predictions = nn.predict(X_test_scaled)
predictions_bool = np.argmax(predictions, axis=1)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions_bool)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions_bool)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions_bool))

69/69 [==============================] - 0s 2ms/step
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,2186,0
Actual 1,5,0


Accuracy Score : 0.9977179370150616
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2186
           1       0.00      0.00      0.00         5

    accuracy                           1.00      2191
   macro avg       0.50      0.50      0.50      2191
weighted avg       1.00      1.00      1.00      2191



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
